In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

import numpy as np
import jax
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int, PRNGKeyArray
import aeon
import pandas as pd

from features.sig_trp import SigVanillaTensorizedRandProj, SigRBFTensorizedRandProj
from features.sig import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureTransformer, TabularTimeseriesFeatures, RandomGuesser
from features.sig_neural import RandomizedSignature
from utils.utils import print_name, print_shape

from preprocessing.timeseries_augmentation import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from aeon.regression.sklearn import RotationForestRegressor
from sklearn.metrics import root_mean_squared_error

jax.config.update('jax_platform_name', 'cpu') # Used to set the platform (cpu, gpu, etc.)
np.set_printoptions(precision=3, threshold=5) # Print options

from aeon.datasets.tser_datasets import tser_soton
from aeon.datasets import load_regression

from sklearn.linear_model import RidgeCV
from aeon.transformations.collection.convolution_based import Rocket, MultiRocketMultivariate, MiniRocketMultivariate

from run_tser import do_experiments, get_aeon_dataset

/home/nikita/Code/zephyrox/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# print(len(tser_soton))

# for dataset_name in list(tser_soton)[10:]:
#     print(dataset_name)
#     X_train, y_train, X_test, y_test = get_aeon_dataset(dataset_name, extract_path="/home/nikita/hdd/Data/TSER/")
#     print_shape(X_train)
#     print_name(y_train)
#     print("\n")

# Load results

In [3]:
# Define the attributes and methods
attributes = ["RMSE_train", "RMSE_test", "time_transform", "time_fit", "alpha"]
methods = ["ridge", "rotforest"]
data_dir = "/home/nikita/Code/zephyrox/Data/"

# Load and store the DataFrames for each attribute and method
dfs = {}
for attribute in attributes:
    for method in methods:
        filename = f"TESR_{attribute}_{method}_results.pkl"
        df = pd.read_pickle(data_dir + filename)
        dfs[(attribute, method)] = df

# Access the loaded DataFrames
# For example, to access the DataFrame for attribute "RMSE_test" and method "ridge":
df_ridge = dfs[("RMSE_test", "ridge")]
        

In [4]:
dfs[("RMSE_test", "rotforest")]

,Random Guesser,Tabular,Sig,Log Sig,Randomized Signature,TRP,RBF TRP,concat TRP,concat RBF TRP,Rocket,MiniRocket,MultiRocket
Covid3Month,1.127887,1.081516,1.098091,1.222497,1.059438,1.189910,1.032519,1.107355,1.055916,1.052265,1.095247,1.089585
AcousticContaminationMadrid,0.974453,0.766033,0.737333,0.792322,0.998104,0.718865,0.545740,0.705139,0.539109,0.711256,0.625820,0.591852
PrecipitationAndalusia,1.225044,0.785274,0.887658,0.882623,1.216886,1.020884,1.032101,0.885010,0.955722,0.937124,0.823828,0.829683
SierraNevadaMountainsSnow,0.941822,0.372057,0.215525,0.359876,0.890480,0.345442,0.360910,0.349584,0.357088,0.606187,0.305829,0.326805
IronConcentration,1.059426,0.768359,1.019163,1.010373,1.080498,1.042788,1.049796,1.074724,1.059827,0.884093,0.903989,0.889800
Covid19Andalusia,0.933270,0.476138,0.447018,0.418603,0.673103,0.621741,0.504709,0.472198,0.544565,0.437018,0.485206,0.537448
EthereumSentiment,1.062330,0.903042,0.915415,0.923742,1.067808,0.927639,0.923669,0.977961,0.977001,1.046174,0.879001,0.884132
ZincConcentration,1.805860,1.743744,1.817797,1.812821,1.841196,1.830751,1.809171,1.838618,1.807798,1.766925,1.735625,1.729620
BinanceCoinSentiment,1.067618,1.011080,1.073592,1.089447,1.052449,1.021535,1.078054,1.063640,1.048617,1.088136,1.046212,1.020177
ManganeseConcentration,1.000994,0.771549,0.924497,0.890578,1.077122,0.881409,0.874210,0.967171,0.883601,0.837819,0.803390,0.823666


In [5]:
dfs[("RMSE_test", "ridge")]
# print(df_rank)

,Random Guesser,Tabular,Sig,Log Sig,Randomized Signature,TRP,RBF TRP,concat TRP,concat RBF TRP,Rocket,MiniRocket,MultiRocket
Covid3Month,1.112402,1.080698,1.088762,1.086677,1.108198,1.109237,1.102124,1.100436,1.098344,1.064213,1.070348,1.073116
AcousticContaminationMadrid,0.973789,0.835874,0.725271,0.798311,0.971189,0.711414,0.884588,0.629135,0.704955,0.692926,0.667704,0.559970
PrecipitationAndalusia,1.230766,0.769093,0.868712,0.904943,1.231266,1.036281,0.850693,0.957740,0.812457,0.862088,0.775117,0.860705
SierraNevadaMountainsSnow,0.891337,0.310508,0.301631,0.369309,0.888828,0.571716,0.407657,0.373019,0.340076,0.634633,0.320543,0.353224
IronConcentration,1.057606,0.764586,1.057118,1.057412,1.037833,1.045872,1.009059,1.038967,1.000327,0.896393,0.855131,0.914020
Covid19Andalusia,0.935700,0.828373,0.874863,0.840894,0.976600,1.139048,0.837848,0.831234,0.629397,0.542943,0.520022,0.462102
EthereumSentiment,1.066777,0.901004,0.889612,0.907614,1.067793,0.950309,0.993861,0.905245,0.926292,1.055517,0.907925,0.903930
ZincConcentration,1.795592,1.699499,1.761020,1.764157,1.796278,1.778420,1.782645,1.774777,1.770304,1.784904,1.749530,1.736234
BinanceCoinSentiment,1.059874,0.997905,1.011165,1.005728,1.096532,1.027420,0.985634,0.991359,1.017788,1.061032,0.995880,1.016473
ManganeseConcentration,0.984627,0.752338,0.852563,0.865637,1.033785,0.860867,0.915614,0.859356,0.897157,0.769229,0.785738,0.824844


In [6]:
(dfs[("RMSE_test", "rotforest")]-dfs[("RMSE_test", "ridge")]).mean()

Random Guesser          0.009232
Tabular                -0.044739
Sig                    -0.239587
Log Sig                -0.146516
Randomized Signature    0.052098
TRP                    -0.313893
RBF TRP                -0.056090
concat TRP             -0.351532
concat RBF TRP          0.001295
Rocket                 -0.025443
MiniRocket             -0.033861
MultiRocket            -0.045376
dtype: float64

In [7]:
dfs[("RMSE_test", "rotforest")].mean()

Random Guesser          1.038081
Tabular                 0.791797
Sig                     0.820580
Log Sig                 0.829411
Randomized Signature    1.076003
TRP                     0.885831
RBF TRP                 0.885247
concat TRP              0.871429
concat RBF TRP          0.861119
Rocket                  0.857781
MiniRocket              0.790244
MultiRocket             0.784138
dtype: float64

In [8]:
dfs[("RMSE_test", "ridge")].mean()

Random Guesser          1.028849
Tabular                 0.836535
Sig                     1.060166
Log Sig                 0.975927
Randomized Signature    1.023906
TRP                     1.199724
RBF TRP                 0.941337
concat TRP              1.222961
concat RBF TRP          0.859824
Rocket                  0.883225
MiniRocket              0.824105
MultiRocket             0.829513
dtype: float64

In [9]:
dfs[("RMSE_test", "ridge")].rank(ascending=True, method='first', axis=1).mean(axis=0)


Random Guesser          9.897959
Tabular                 5.040816
Sig                     5.775510
Log Sig                 6.979592
Randomized Signature    9.632653
TRP                     8.265306
RBF TRP                 6.979592
concat TRP              7.163265
concat RBF TRP          5.306122
Rocket                  5.510204
MiniRocket              3.346939
MultiRocket             4.102041
dtype: float64

In [10]:
dfs[("RMSE_train", "ridge")].rank(ascending=True, method='first', axis=1).mean(axis=0)


Random Guesser          10.938776
Tabular                  6.265306
Sig                      6.428571
Log Sig                  8.142857
Randomized Signature    10.204082
TRP                      8.224490
RBF TRP                  5.183673
concat TRP               7.795918
concat RBF TRP           4.408163
Rocket                   4.591837
MiniRocket               2.877551
MultiRocket              2.938776
dtype: float64

In [11]:
dfs[("RMSE_test", "rotforest")].rank(ascending=True, method='first', axis=1).mean(axis=0)

Random Guesser           9.163265
Tabular                  4.591837
Sig                      5.734694
Log Sig                  6.081633
Randomized Signature    10.530612
TRP                      7.612245
RBF TRP                  7.102041
concat TRP               7.142857
concat RBF TRP           6.551020
Rocket                   5.836735
MiniRocket               3.857143
MultiRocket              3.795918
dtype: float64

In [12]:
(dfs[("RMSE_test", "rotforest")].rank(ascending=True, method='first', axis=1) == 1).sum()

Random Guesser           5
Tabular                 13
Sig                      6
Log Sig                  2
Randomized Signature     1
TRP                      1
RBF TRP                  2
concat TRP               1
concat RBF TRP           2
Rocket                   2
MiniRocket               5
MultiRocket              9
dtype: int64